In [ ]:
%%bash
nrnivmodl

In [ ]:
# Download NEURON: http://www.neuron.yale.edu/neuron/download
# Download PyNeuronToolbox: https://github.com/ahwillia/PyNeuron-Toolbox

from neuron import h
import numpy as np
import math
import pylab as plt
import scipy.linalg
from PyNeuronToolbox.record import ez_record,ez_convert
from PyNeuronToolbox.morphology import shapeplot,shapeplot_animate
from mpl_toolkits.mplot3d import Axes3D
from JSAnimation import IPython_display
from matplotlib import animation
from matplotlib.pyplot import cm
import warnings
warnings.filterwarnings('ignore', category=SyntaxWarning)
%matplotlib inline
np.random.seed(123456789)

DC = 10 # diffusion coeff [microns^2 s^-1]

# Load morphology and other stuff
# --> SegLists: soma[2], dend[74], dend_5[37], apic[42], axon[1]
# --> Files from Migliore & Migliore (2012)
# --> CA1 pyramidal neuron
h.load_file('stdrun.hoc')
h.xopen('ri06.hoc')
h.xopen('fixnseg.hoc')
h.xopen('5a_nogui.hoc')
h.tstop = 700.0

In [ ]:
from PyNeuronToolbox.morphology import dist_between,allsec_preorder

def dist_to_soma(segment):
    return dist_between(h,h.soma[0](0.5),segment)

#seglist in pre-order
sec_list = allsec_preorder(h)
seg_list = []
for sec in sec_list:
    locs = np.linspace(0,1,sec.nseg+2)[1:-1]
    for loc in locs:
        seg_list.append(sec(loc))
n = len(seg_list)

In [ ]:
for i in range(len(sec_list)):
    sec = sec_list[i]
    print(f"{i}: {sec.name()}, nseg={sec.nseg}, L={sec.L}, or={sec.orientation()}, hname={sec.hname()}")

In [ ]:
(dist_to_soma(sec(0)),dist_to_soma(sec(1)),sec.L,np.linspace(0,1,sec.nseg+2),np.linspace(0,1,sec.nseg+2)[1:-1])

In [ ]:
dts = [dist_to_soma(s) for s in seg_list]
dts[:11]

In [ ]:
import re
import pandas as pd
p = re.compile("^([^\\[]+)\\[.+")
tps = [p.sub("\\1",seg.sec.name()) for seg in seg_list]
pd.Series(tps).drop_duplicates().tolist()

In [ ]:
def sushi_system(a,b,c,d,l):
    """
    Returns a matrix A, such that dx/dt = A*x
    
    N = # of compartments
    A is (2N x 2N) matrix
    x is (2N x 1) vector.
      The first N elements correspond to concentrations of u (molecules in transit)
      The second half correspond to concentrations of u-star (active molecules)
    The trafficking rate constants along the microtubules are given by the vectors "a" and "b"
    The rate constants for u turning into u* is given by the vector "c"
    The rate constants for u* turning into u is given by the vector "d"
    The rate constants for the degradation of u* is given by the vector "l"
    """
    # number of compartments
    N = len(l)
    
    ## State-space equations
    #  dx/dt = Ax + Bu
    A = np.zeros((2*N,2*N))

    # Trafficking along belt
    # Iterative traversal of dendritic tree in pre-order
    i = 0
    section = None
    parentStack = [(None,h.soma[0])]
    while len(parentStack)>0:
        # Get next section to traverse
        #  --> p is parent index, section is h.Section object
        (p,section) = parentStack.pop()
        
        # Trafficking to/from parent
        if p is not None:
            # Out of parent, into child
            ai = a.pop()
            A[p,p] += -ai
            A[i,p] += ai
            # Into parent, out of child
            bi = b.pop()
            A[p,i] += bi
            A[i,i] += -bi
        
        # visit all segments in compartment
        for (j,seg) in enumerate(section):
            # Deal with out/into rates within compartment, just tridiag matrix
            if j>0:
                # Out of parent, into child
                ai = a.pop()
                A[i-1,i-1] += -ai
                A[i,i-1] += ai
                # Into parent, out of child
                bi = b.pop()
                A[i-1,i] += bi
                A[i,i] += -bi
            # move onto next compartment
            i += 1
        
        # now visit children in pre-order
        child_list = list(h.SectionRef(sec=section).child)
        if child_list is not None:
            child_list.reverse()
        for c_sec in child_list:
            parentStack.append([i-1,c_sec]) # append parent index and child
    
    # Trafficking off the belt
    for i in range(N):
        A[i,i] += -c[i]
        A[i+N,i] += c[i]
    
    # Reattachment to belt
    #for i in range(N):
    #    # reattachment
    #    A[i, i + N] += d[i]
    #    A[i + N, i + N] += -d[i]

    # Degradation after being taken off the belt
    for i in range(N):
        A[i+N,i+N] = -l[i]
    
    return A

In [ ]:
def trafficking_solution(utarg):
    """ Solve the problem by tuning trafficking rates, like Figs 1 and 2. """
    x = []
    
    # Iterative traversal of dendritic tree in pre-order
    i = 0
    section = None
    parentStack = [(None,h.soma[0])]
    while len(parentStack)>0:
        # Get next section to traverse
        #  --> p is parent index, section is h.Section object
        (p,section) = parentStack.pop()
        
        # Trafficking to/from parent
        if p is not None:
            mp = utarg[p] # concentration in parent
            mc = utarg[i] # concentration in child
            x.insert(0,mp/mc)
        
        # visit all segments in compartment
        for (j,seg) in enumerate(section):
            # Deal with out/into rates within compartment, just tridiag matrix
            if j>0:
                mp = utarg[i-1]
                mc = utarg[i]
                x.insert(0,mp/mc)
                
            # move onto next compartment
            i += 1
        
        # now visit children in pre-order
        child_list = list(h.SectionRef(sec=section).child)
        if child_list is not None:
            child_list.reverse()
        for c_sec in child_list:
            parentStack.append([i-1,c_sec]) # append parent index and child
    
    # return calculated guesses (flip, up/down since get_deriv pops from start)
    return np.array(x)

def get_sys_matrix(utarg,F=0.5,Ctau=1e-3,dscale=0.1,dv=1e-7):
    # F is a mixing factor between 0 and 1
    K = np.sum(utarg)/n
    x = trafficking_solution(F*utarg + (1-F)*K)
    a = (1/(1+x))
    a = list(a)
    b = list((1/(1+x**-1)))
    l = list(np.ones(n)*dv)
    c = list(Ctau*utarg/(F*utarg + (1-F)*K))
    d = list([ci * dscale for ci in c])
    A = sushi_system(a,b,c,d,l)
    return A

In [ ]:
# matrix exponential used to solve linear system
from scipy.linalg import expm

# initial condition, u starts only in soma and is trafficked out to dendrites
u0 = np.zeros(2*n)
u0[0] = 1.0  # compartment 0 is the soma

In [ ]:
def solve_u(u0,w,V,Vinv,t):
    D = np.diag(np.exp(w*t))          # diagonal matrix exponential
    PHI = np.real(V.dot(D.dot(Vinv))) # state transition matrix
    return PHI.dot(u0)                # calculate u(t)

def sim_time(A,u0,time,nframes=10):
    # Run a simulation (log time)
    # --> this is a linear system; thus, matrix exponential provides exact solution
    utrace = [u0]
    w,V = scipy.linalg.eig(A)
    Vinv = np.linalg.inv(V)
    t = np.logspace(-0.5,math.log10(time),nframes)
    for t_ in t: utrace.append(solve_u(u0,w,V,Vinv,t_))
    return np.array(utrace).T


def run_sim(A,nframes=10):
    # Run a simulation (log time)
    # --> this is a linear system; thus, matrix exponential provides exact solution
    utrace = [u0]
    t = np.logspace(-0.5,8.5,nframes)
    w,V = scipy.linalg.eig(A)
    Vinv = np.linalg.inv(V)
    t = np.logspace(-0.5,8.5,nframes)
    for t_ in t: utrace.append(solve_u(w,V,Vinv,t_))
    return np.array(utrace).T

def animate_sim(climits=[0,0.0035]):
    u = utrace[n:,:]
    fig = plt.figure(figsize=(8,8))
    shapeax = plt.subplot(111, projection='3d')
    lines = shapeplot(h,shapeax,order='pre',lw=2)
    shapeax.view_init(elev=80,azim=-90)
    plt.title('distribution of active/detached cargo',fontweight='bold',fontsize=14)
    plt.axis('off')

    # Hack to get a colorbar
    # http://stackoverflow.com/questions/8342549/matplotlib-add-colorbar-to-a-sequence-of-line-plots
    sm = plt.cm.ScalarMappable(cmap=plt.cm.cool, norm=plt.Normalize(climits[0],climits[1])) 
    #sm._A = []
    #plt.colorbar(sm, shrink=0.5)
    #plt.colorbar(sm)
    plt.tight_layout()

    anim_func = shapeplot_animate(u.T,lines,u.shape[1],clim=climits,cmap=cm.cool)
    return animation.FuncAnimation(fig, anim_func, frames=u.shape[1], interval=5e2, blit=True)

def snap_ss(name,clim=[0,0.0035],cmap=plt.cm.cool):
    u = utrace[n:,:]
    fig = plt.figure(figsize=(8,8))
    shapeax = plt.subplot(111, projection='3d')
    lines = shapeplot(h,shapeax,cvals=u[:,-1],clim=clim,\
                      cmap=cmap,order='pre',lw=2)
    shapeax.view_init(elev=80,azim=-90)
    plt.title('steady-state',fontweight='bold',fontsize=14)

    # colorbar
    sm = cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(clim[0],clim[1])) 
    #sm = plt.cm.ScalarMappable(cmap=plt.cm.cool, norm=plt.Normalize(clim[0],clim[1])) 
    #sm._A = []
    #plt.colorbar(sm, shrink=0.5)
    #plt.colorbar(sm,cax=shapeax)
    #plt.colorbar()
    plt.tight_layout()
    plt.axis('off')
    #plt.savefig(name+'.svg')

Read Experimental Data
----

In [ ]:
tdf=pd.read_csv('../data/seg_mapping.csv')
abbCA1=tdf['abb']
abbT={}
segIdx={}
for i in range(n):
    abbT[abbCA1[i]] = 1+ abbT.get(abbCA1[i],0)
    ll=segIdx.get(abbCA1[i],[])
    ll.append(i)
    segIdx[abbCA1[i]] = ll

(abbCA1,abbT,segIdx.keys(),{k:len(segIdx.get(k)) for k in segIdx.keys()})

In [ ]:
expD=pd.read_csv('../data/CA1_gradient.csv')
subreg = ['CA1so', 'CA1sr', 'CA1slm']
expD

In [ ]:
import sys
sys.path.append('../')
import sushibelt
import time

In [ ]:
utarg = np.ones(len(seg_list))
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    utarg[sidx] *= i
u1 = np.concatenate((utarg,utarg))
utrace = [u1,u1]
utrace = np.array(utrace).T
snap_ss(f"segment_distribution",clim=[0,max(utarg)],cmap=plt.cm.YlOrBr)
max(utarg)

In [ ]:
utarg = np.ones(len(seg_list))
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    
    sidx= segIdx[abb]
    utarg[sidx] *= [j for j in range(len(subreg)) if subreg[j]== expD['Subregion'][i]][0]
u1 = np.concatenate((utarg,utarg))
utrace = [u1,u1]
utrace = np.array(utrace).T
snap_ss(f"subregion_distribution",clim=[0,max(utarg)],cmap=plt.cm.YlOrBr)
max(utarg)

Day 0 3 weeks
------

In [ ]:
cname='D0W3'
utarg = np.ones(len(seg_list))
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    utarg[sidx] *= expD[f"{cname}_MEAN"][i]/len(sidx)
u1 = np.concatenate((utarg,utarg))
utrace = [u1,u1]
utrace = np.array(utrace).T
max(utarg)

In [ ]:
x=np.array(range(expD.shape[0]),dtype=int)
nobsD0 = np.array(expD[f"{cname}_MEAN"])#/np.sum(expD[f"{cname}_MEAN"])
plt.plot(x,nobsD0,label=cname)
plt.show()

In [ ]:
cname='D7W3'
obsD = np.ones(len(seg_list))
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    obsD[sidx] *= expD[f"{cname}_MEAN"][i]/len(sidx)
nobsD = np.array(expD[f"{cname}_MEAN"])#/np.sum(expD[f"{cname}_MEAN"])
uD = np.concatenate((obsD,obsD))
utrace = [uD,uD]
utrace = np.array(utrace).T
max(obsD)

In [ ]:
x=np.array(range(expD.shape[0]),dtype=int)

plt.plot(x,nobsD0,label=cname)
plt.plot(x,nobsD,label='detached DC')
plt.show()

In [ ]:
diF=abs(uD-u1)
utrace = [diF,diF]
utrace = np.array(utrace).T
(min(diF),max(diF))

In [ ]:
cname='D7M3'
obsD = np.ones(len(seg_list))
resMo = np.zeros(expD.shape[0])
resFo = np.zeros(expD.shape[0])
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    obsD[sidx] *= expD[f"{cname}_MEAN"][i]
    resMo[i] = np.sum(utrace[sidx,-1])
    resFo[i] = np.sum(utrace[[j+n for j in sidx],-1])
nobsD = np.array(expD[f"{cname}_MEAN"])/np.sum(expD[f"{cname}_MEAN"])
nobsDnn = np.array(expD[f"{cname}_MEAN"])/np.sum(expD["D0M3_MEAN"])
nobsDsd = np.array(expD[f"{cname}_SD"])/np.sum(expD["D0M3_MEAN"])
nresM = resMo/np.sum(resMo)
nresF = resFo/np.sum(resFo)
(nobsD,nresM,nresF,nobsDnn,nobsDsd)

In [ ]:
itarg = np.ones(len(seg_list), dtype=int)
par = [1000,0,0,0,1,2,3,4]
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx = segIdx[abb]
    itarg[sidx] *= [j + 3 for j in range(len(subreg)) if subreg[j] == expD['Subregion'][i]][0]
utarg = [par[j] for j in itarg]


In [ ]:
bgSignal = 1e-5
day7 = 7 * 24 * 3600 # final time point

def calcUtrace(par,delta=bgSignal):
    F = par[0]
    Ctau = 10 ** par[1]
    mProp = par[2]
    dv = np.zeros(n)
    utarg = delta*np.ones(n)
    for k in range(n):
        if itarg[k] > 2:
            dv[k] = 10 ** par[itarg[k]]
            utarg[k] = par[itarg[k]+3]
    utarg /= np.sum(utarg)
    K = np.sum(utarg) / n
    x = trafficking_solution(F * utarg + (1 - F) * K)
    a = (1 / (1 + x))
    a = list(a)
    b = list((1 / (1 + x ** -1)))
    l = list(dv)
    c = list(Ctau * utarg / (F * utarg + (1 - F) * K))
    d = list(np.zeros(n))
    A = sushi_system(a, b, c, d, l)
    u0 = np.concatenate((mProp * dinit, (1 - mProp) * dinit))
    utrace = sim_time(A, u0, day7)
    return utrace

cname='D0M3'
d0w = np.ones(n)
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    d0w[sidx] *= expD[f"{cname}_MEAN"][i]/len(sidx)
dinit = d0w/np.sum(d0w)
#par = [0.29465728385098655,-2.46803964145224,0.586395975556826,-5.286521134624552,-5.408070983691747,-5.779005887950339,0.9999999999999953,0.427256332241015,0.001]
#par = [0.29465728385098655,-2.46803964145224,0.5863959753760672,-5.286521134642408,-5.408070983925882,-5.7790058879511825,0.9999999999999928,0.4272563322410141,0.001]
#cost=0.03871
#par = [0.0,-2.847372550934951,0.20430623530833164,-5.754630055624563,-5.8321723784857635,-6.0649088262203215,0.9999999999999993,0.9550522528085269,0.001]
#cost = 0.00010140231658948862
#par = [0.0,-2.847372550934951,0.20430623530833114,-5.754630053000959,-5.832172379474677,-6.064908828218014,1.0,0.955052252808502,0.001]
#cost = 0.00010140231658946115
par = [0.999012635491613,-9.361703560541367,0.9976298142299469,-5.748708930176829,-5.897369376187479,-6.065761766702373,0.9999999999999993,0.9999999999999993,0.0010000000038804917]
cost = 0.00015575301605738785
mProp = par[2]
utrace = calcUtrace(par)
resM, resF = sushibelt.aggregate_segments(utrace[:, -1], segIdx, expD['Abbreviation'], fun=np.sum)


In [ ]:
{'F' : par[0],
'Ctau' : 10 ** par[1],
'mProp' : par[2],
'dv_CA1so': 10 ** par[3],
'dv_CA1sr': 10 ** par[4],
'dv_CA1slm': 10 ** par[5],
'demand_CA1so': par[6],
'demand_CA1sr': par[7],
'demand_CA1slm': par[8],
'sum_nobsDnn' : np.sum(nobsDnn),
'sum_nobsDnn^2' : np.sum(nobsDnn ** 2),
'sum_resM' : np.sum(resM),
'sum_resF' : np.sum(resF),
'sum_tot' : np.sum(resF)+np.sum(resM),
'err' : np.sum((resF/(1-mProp) - nobsDnn) ** 2),
'err_norm' : np.sum((resF/(1-mProp) - nobsDnn) ** 2)/np.sum(nobsDnn ** 2),
'chi2' : np.sum(((resF/(1-mProp) - nobsDnn)/nobsDsd) ** 2),
'cost' : cost
}

In [ ]:
Nvals=10
([1.0 * i/Nvals for i in range(Nvals+1)],(resF/(1-mProp) - nobsDnn),(resF/(1-mProp) - nobsDnn)/nobsDsd)

In [ ]:
parnames=['F','Ctau','dv','mProp','demand_CA1so','demand_CA1sr','demand_CA1slm']
Nvals = 4
parvals = [1.0 * i/Nvals for i in range(Nvals+1)]
numPar=len(parnames)
lowb=np.array([0, -18, -18, 1e-3, 1e-3, 1e-3, 1e-3])
upbga=np.array([1, -1, -1, 1, 1, 1, 1])

def prepPar(cpar,pn,pval,pidx,bidx):
    lpar = np.zeros(numPar)
    lpar[pidx] = pval
    for j in range(numPar - 1):
        lpar[bidx[j]] = cpar[j]
    return lpar

i = 0
j = 0
pn, pv = (parnames[i],parvals[j])
pidx=[i for i in range(numPar) if parnames[i]==pn][0]
pval=lowb[pidx]+(upbga[pidx]-lowb[pidx])*pv
bidx= [i for i in range(numPar) if i != pidx]
clowb=lowb[bidx]
cupbga=upbga[bidx]
cpar = [i+0.1 for i in range(numPar-1)]
lpar = prepPar(cpar,pn,pval,pidx,bidx)
{'numPar' : numPar,
 'Nvals' : Nvals,
 'parvals' : parvals,
 'pn' : pn,
 'pv' : pv,
 'pidx' : pidx,
 'pval' : pval,
 'bidx' : bidx,
 'cpar' : cpar,
 'lpar' : lpar
}

In [ ]:
pDF = pd.DataFrame(lpar).T
pDF.index = [55]
pDF

In [ ]:
x=np.array(range(expD.shape[0]),dtype=int)

plt.plot(x,nobsDnn,label='Target')
#plt.plot(x,nresM,label='mobile O')
#plt.plot(x,nresF,label='detached O')
plt.plot(x,resM,label='mobile DC')
plt.plot(x,resF,label='detached DC')
plt.ylabel('Value')
plt.xlabel('Region number')
plt.yscale("log")
# show a legend on the plot
plt.legend()
plt.title(f"Cost = {cost}") 
# function to show the plot
plt.show()
#plt.savefig('bestFitCA1_3dv_20240603.png')
#plt.savefig('bestFitCA1_3dv_20240603.pdf')
#plt.savefig('bestFitCA1_3dv_20240603.svg')


In [ ]:
fig, ax = plt.subplots()
ax.plot(x,nobsDnn,'o-',label='D7M3')
#plt.plot(x,nresM,label='mobile O')
#plt.plot(x,nresF,label='detached O')
ax.plot(x,resF/(1-mProp),label='detached norm DC')
ax.set_xlabel('Region number')
ax.set_ylabel('Value')
ax.set_xticks(x,expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend()
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
#plt.title(f"Cost = {cost}") 
plt.title(f"Cost = {format(cost,'.4g')}, $\chi^2$ = {format(np.sum(((resF/(1-mProp) - nobsDnn)/nobsDsd) ** 2),'.4g')}") 
# function to show the plot
plt.show()
#plt.savefig('bestFitCA1_3dv_20240603.fit.png')
#plt.savefig('bestFitCA1_3dv_20240603.fit.pdf')
#plt.savefig('bestFitCA1_3dv_20240603.fit.svg')


In [ ]:
fig, ax = plt.subplots()
ax.plot(x,resM,label='mobile DC')
ax.plot(x,resF,label='detached DC')
ax.set_yscale("log")
ax.set_xlabel('Region number')
ax.set_ylabel('Value')
ax.set_xticks(x,expD['Abbreviation'],rotation=90)
# show a legend on the plot
ax.legend()
pos = ax.get_position()
ax.set_position([pos.x0, pos.y0 + pos.height*0.15, pos.width , pos.height* 0.85])
#plt.title(f"Cost = {cost}") 
plt.title(f"Cost = {format(cost,'.4g')}, $\chi^2$ = {format(np.sum(((resF/(1-mProp) - nobsDnn)/nobsDsd) ** 2),'.4g')}") 
# function to show the plot
plt.show()

In [ ]:
F = par[0]
Ctau = 10 ** par[1]
mProp = par[2]
dv = np.zeros(n)
utarg = bgSignal*np.ones(n)
for k in range(n):
    if itarg[k] > 2:
        dv[k] = 10 ** par[itarg[k]]
        utarg[k] = par[itarg[k]+3]
    else:
        dv[k] = np.nan
        utarg[k] = np.nan
#utarg /= np.sum(utarg)
bfit, afit = np.polyfit(par[6:], 10 ** np.array(par[3:6]), deg=1)
xseq = np.linspace(min(par[6:]), max(par[6:]), num=100)
#plt.scatter(utarg,dv)
fig, ax = plt.subplots(figsize = (9, 9))
ax.scatter(par[6:], 10 ** np.array(par[3:6]))
ax.plot(xseq, afit + bfit * xseq, color="k", lw=2.5)
pos = ax.get_position()
#ax.text(pos.x0, pos.y0 + pos.height, f'LR equation: $Y = {round(afit*1e6,3)}'+'\\ 10^{-6}'+f' + {round(bfit*1e6,3)}'+'\\ 10^{-6}\\ X$', fontsize=10)
ax.text(0.2,1.85e-6, f'LR equation: $Y = {round(afit*1e6,3)}'+'\\ 10^{-6}'+f' + {round(bfit*1e6,3)}'+'\\ 10^{-6}\\ X$', fontsize=10)
plt.xlabel('Demand value')
plt.ylabel('Degradation rate')
plt.show()
#plt.savefig('bestFitCA1_3dv_scatter_20240601.png')
#plt.savefig('bestFitCA1_3dv_scatter_20240601.pdf')
#plt.savefig('bestFitCA1_3dv_scatter_20240601.svg')


In [ ]:
(xseq, afit + bfit * xseq,pos)

In [ ]:
bfit, afit = np.polyfit(par[6:], 10 ** np.array(par[3:6]), deg=1)
plt.plot(xseq, afit + bfit * xseq, color="k", lw=2.5)
(bfit, afit,f'LR equation: $Y = {round(afit*1e6,3)}'+'\\ 10^{-6}'+f' + {round(bfit*1e6,3)}'+'\\ 10^{-6}\\ X$')

Day 7 3 weeks
----

In [ ]:
cname='D7W3'
utarg = np.ones(len(seg_list))
for i in range(expD.shape[0]):
    abb = expD['Abbreviation'][i]
    sidx= segIdx[abb]
    utarg[sidx] *= expD[f"{cname}_MEAN"][i]
u1 = np.concatenate((utarg,utarg))
utrace = [u1,u1]
utrace = np.array(utrace).T
snap_ss(f"case1_{cname}_target",clim=[0,max(utarg)],cmap=plt.cm.YlOrBr)
max(utarg)